In [10]:
import numpy as np 
import math as m 
import timeit 

def CRR_Tree(S,v,n,T): 
    h = T/n    # time step
    u = np.exp( v * np.sqrt(h))            # u = e ^ (v * sqrt(h)), v = volatility (sigma )
    d = 1/u   
    Tree = np.zeros((n+1, n+1))            # setup: n steps trees have n + 1 nodes 
    
    for j in range(n+1):   # 分成幾個 step : columns
        for i in range(j+1):  # 有幾個j就有幾個i  # i: # of down, j: # of total moves , j-i : # of up
            Tree[i,j] = S * m.pow(d,i) * m.pow(u,j-i) 
    
    print('n = ' + str(n) + ':\n',np.matrix(np.round(Tree,2)))         
    return Tree

In [11]:
S = 40
v = 0.3
n = 5 
T = 1 
x = CRR_Tree(S,v,n,T)

n = 5:
 [[40.   45.74 52.31 59.82 68.41 78.23]
 [ 0.   34.98 40.   45.74 52.31 59.82]
 [ 0.    0.   30.59 34.98 40.   45.74]
 [ 0.    0.    0.   26.75 30.59 34.98]
 [ 0.    0.    0.    0.   23.39 26.75]
 [ 0.    0.    0.    0.    0.   20.45]]


### Check 

In [12]:
h = T/n    # time step
u = np.exp( v * np.sqrt(h))    # u = e ^ (v * sqrt(h))
d = 1/u   

Tree = np.zeros((n+1, n+1))   # setup: n steps trees have n + 1 nodes 
Tree

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [13]:
# i: # of down, j: # of total moves , j-i : # of up 
import time 

for j in range(n+1): 
        for i in range(j+1): 
            Tree[i,j] = S * m.pow(d,i) * m.pow(u,j-i) 
                            # d**i * u**j-i
            print('n = ' + str(n) + ':\n',np.matrix(np.round(Tree,2)))    
            time.sleep(1)     


n = 5:
 [[40.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
n = 5:
 [[40.   45.74  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5:
 [[40.   45.74  0.    0.    0.    0.  ]
 [ 0.   34.98  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5:
 [[40.   45.74 52.31  0.    0.    0.  ]
 [ 0.   34.98  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5:
 [[40.   45.74 52.31  0.    0.    0.  ]
 [ 0.   34.98 40.    0.    0.    0.  ]
 [ 0.    0.   

### Pricing European Call 

In [17]:
import numpy as np
import math as m

def BinoEuroCall(S, K, v, r, n, T):
    # Calculate time step
    h = T / n

    # Calculate up and down factors
    u = m.exp(v * np.sqrt(h))
    d = 1 / u    # CRR assumption 

    # Calculate risk-neutral probability ( nomoral q )
    q = (np.exp(r * h) - d) / (u - d)                  # q = R-d / (u-d)  or q = (e ^ (rh) - d) / u - d 

    
    # CRR 
    Tree = np.zeros((n + 1, n + 1))    
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    
    
    # Call Price Tree 
    Call = np.zeros((n + 1, n + 1))
    
    for j in range(n + 1, 0, -1):    # backward induction : from n+1 to 0
        for i in range(j):  
            
            if j == n + 1:   # when j = n+1, we're at T. calculate terminal payoff  # a specific node 
                # At maturity, payoff is max(Tree - K, 0)
                Call[i, j - 1] = max(Tree[i, j - 1] - K, 0)   
            else:
                # Discounted expected value for intermediate nodes
                Call[i, j - 1] = np.exp(-r * h) * (q * Call[i, j] + (1 - q) * Call[i + 1, j])

    # Print stock price tree and option value tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    print('-------------------------------------------')
    print('Option Value Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2)))

    
    # result 
    return Call[0, 0]


In [18]:
S = 100
K = 100 
r = .05 
v = .3 
T = 20/36 
n = 5 
cp = BinoEuroCall(S,K,v,r,n,T) 
print('The binomial European call price is ' + str(round(cp,6)))   # change the rounding to see more decimal 

Stock Price Tree (n = 5):
 [[100.   110.52 122.14 134.99 149.18 164.87]
 [  0.    90.48 100.   110.52 122.14 134.99]
 [  0.     0.    81.87  90.48 100.   110.52]
 [  0.     0.     0.    74.08  81.87  90.48]
 [  0.     0.     0.     0.    67.03  74.08]
 [  0.     0.     0.     0.     0.    60.65]]
-------------------------------------------
Option Value Tree (n = 5):
 [[10.65 16.56 24.94 36.09 49.74 64.87]
 [ 0.    4.79  8.27 13.95 22.69 34.99]
 [ 0.    0.    1.31  2.63  5.26 10.52]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
The binomial European call price is 10.649966


In [19]:
# just for visualization


h = T / n
u = np.exp(v * np.sqrt(h))
d = 1 / u
q = (np.exp(r * h) - d) / (u - d)   

Tree = np.zeros((n + 1, n + 1))     
for j in range(n + 1):
    for i in range(j + 1):
        Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

Call = np.zeros((n + 1, n + 1))

for j in range(n + 1, 0, -1):    
        for i in range(j):
            
            if j == n + 1:   
                Call[i, j - 1] = max(Tree[i, j - 1] - K, 0)  
            
            else:
                Call[i, j - 1] = m.exp(-r * h) * (q * Call[i, j] + (1 - q) * Call[i + 1, j])  # i: number of downs 
            
            print('n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2))) 
            time.sleep(1)
            

n = 5):
 [[ 0.    0.    0.    0.    0.   64.87]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5):
 [[ 0.    0.    0.    0.    0.   64.87]
 [ 0.    0.    0.    0.    0.   34.99]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5):
 [[ 0.    0.    0.    0.    0.   64.87]
 [ 0.    0.    0.    0.    0.   34.99]
 [ 0.    0.    0.    0.    0.   10.52]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5):
 [[ 0.    0.    0.    0.    0.   64.87]
 [ 0.    0.    0.    0.    0.   34.99]
 [ 0.    0.    0.    0.    0.   10.52]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
n = 5):
 [[ 0.    0.    

### European Put 

In [20]:
import numpy as np
import math as m

def BinoEuroPut(S, K, v, r, n, T):
    ### Underlying asset tree 
    h = T / n

    u = np.exp(v * np.sqrt(h))
    d = 1 / u
    q = (m.exp(r * h) - d) / (u - d)   

    Tree = np.zeros((n + 1, n + 1))     ## CRR tree reamins the same 
    
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    
    
    ### Put Tree 
    Put = np.zeros((n + 1, n + 1))
    
    for j in range(n + 1, 0, -1):    
        for i in range(j):
            
            if j == n + 1:   
                Put[i, j - 1] = max(K - Tree[i, j - 1], 0)  # Put : max(K - St, 0 ) 
            else:
                Put[i, j - 1] = m.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j])  # i : number of downs 

    # Print stock price tree and option value tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    print('-------------------------------------------')
    print('Option Value Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Put, 2)))

    
    # result 
    return Put[0, 0]


In [21]:
S = 100
K = 100 
r = .05 
v = .3 
T = 20/36    # h = T / n 
n = 5 
CP = BinoEuroCall(S,K,v,r,n,T)   # call price 
print('=======================================================')

PP = BinoEuroPut(S,K,v,r,n,T) 

print('The binomial European put price is ' + str(round(PP,6)))   
print('The binomial European call price is ' + str(round(CP,6)))   

Stock Price Tree (n = 5):
 [[100.   110.52 122.14 134.99 149.18 164.87]
 [  0.    90.48 100.   110.52 122.14 134.99]
 [  0.     0.    81.87  90.48 100.   110.52]
 [  0.     0.     0.    74.08  81.87  90.48]
 [  0.     0.     0.     0.    67.03  74.08]
 [  0.     0.     0.     0.     0.    60.65]]
-------------------------------------------
Option Value Tree (n = 5):
 [[10.65 16.56 24.94 36.09 49.74 64.87]
 [ 0.    4.79  8.27 13.95 22.69 34.99]
 [ 0.    0.    1.31  2.63  5.26 10.52]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
Stock Price Tree (n = 5):
 [[100.   110.52 122.14 134.99 149.18 164.87]
 [  0.    90.48 100.   110.52 122.14 134.99]
 [  0.     0.    81.87  90.48 100.   110.52]
 [  0.     0.     0.    74.08  81.87  90.48]
 [  0.     0.     0.     0.    67.03  74.08]
 [  0.     0.     0.     0.     0.    60.65]]
-------------------------------------------
Option Value Tree (n = 5):
 [[ 7.91  3.85  1.15  0.  

In [34]:
### put call parity: C - P ( long call, short put ) = S - PV( k )
""" 
C : 
"""

In [22]:

CP - PP 

2.7395522883652044

In [23]:
S - K * np.exp(-r * T)

2.739552288365161

### Pricing American Put

In [36]:
import numpy as np
import math as m

def BinoAmerPut(S, K, v, r, n, T, div):  # div : continuous dividend yield --> necessary for american call 
    # Time step size
    h = T / n

    u = m.exp(v * m.sqrt(h))
    d = 1 / u
    q = (m.exp((r - div) * h) - d) / (u - d)  #  e^(r - delta) h / u - d 

    Tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    # Put Tree 
    Put = np.zeros((n + 1, n + 1))
    Exer = np.zeros((n + 1, n + 1))  # check for early exercise 
    
    for j in range(n + 1, 0, -1):
        for i in range(j):
            
            if j == n + 1:
                # Option price at maturity 
                Put[i, j - 1] = max(K - Tree[i, j - 1], 0)  # put: K-St
            
            else:
                Put[i, j - 1] = max(K - Tree[i, j - 1],  # early exercise value of american put 
                                    np.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j]))  # the waiting value: based on European put 


                # for visualizing early exercise 
                if K - Tree[i, j - 1] > m.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j]): 
                    Exer[i, j - 1] = 1 
                    
                    
    # Print stock price tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    
    print('-------------------------------------------')

    # Print option price tree for American put
    print('Option Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Put, 2)))
    
    print('-------------------------------------------')

    print('n = ' + str(n) + '):\n', np.matrix(Exer))
    
    

    # Return the option price at the root node
    return Put[0, 0]


In [37]:
# Parameters
S = 100          # Initial stock price
K = 101          # Strike price
v = 0.2          # Volatility
r = 0.05         # Risk-free interest rate
T = 2            # Time to maturity
n = 5            # Number of steps in the binomial tree
div = 0          # Dividend yield

# Calculate the binomial American put option price
App = BinoAmerPut(S, K, v, r, n, T, div)

# Print the result
print(f'The binomial American put price is {round(App, 3)}')


Stock Price Tree (n = 5):
 [[100.   113.48 128.79 146.15 165.86 188.22]
 [  0.    88.12 100.   113.48 128.79 146.15]
 [  0.     0.    77.65  88.12 100.   113.48]
 [  0.     0.     0.    68.42  77.65  88.12]
 [  0.     0.     0.     0.    60.29  68.42]
 [  0.     0.     0.     0.     0.    53.13]]
-------------------------------------------
Option Price Tree (n = 5):
 [[ 8.38  3.83  1.12  0.    0.    0.  ]
 [ 0.   14.27  7.3   2.53  0.    0.  ]
 [ 0.    0.   23.35 13.41  5.71  0.  ]
 [ 0.    0.    0.   32.58 23.35 12.88]
 [ 0.    0.    0.    0.   40.71 32.58]
 [ 0.    0.    0.    0.    0.   47.87]]
-------------------------------------------
n = 5):
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]]
The binomial American put price is 8.38


In [38]:
App   # result 

8.379617641806195

### American Call 

In [39]:
import numpy as np
import math as m

def BinoAmerCall(S, K, v, r, n, T, div):  # div : continuous dividend yield --> necessary for american call 
    # Time step size
    dt = T / n

    # Up and down factors
    u = m.exp(v * m.sqrt(dt))
    d = 1 / u
    q = (m.exp((r - div) * dt) - d) / (u - d)

    # Initialize stock price tree
    Tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    # Put Tree 
    Call = np.zeros((n + 1, n + 1))
    Exer = np.zeros((n + 1, n + 1)) 
    
    for j in range(n + 1, 0, -1):
        for i in range(j):
            
            if j == n + 1:
                Call[i, j - 1] = max(Tree[i, j - 1] - K , 0)  # Call : St- K 
            
            else:
                Call[i, j - 1] = max(Tree[i, j - 1] - K ,  # early exercise value of american call 
                                    m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]))  # the waiting value: based on European call

            
                if Tree[i, j - 1] - K  > m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]): 
                    Exer[i, j - 1] = 1 
                    
                    
    # Print stock price tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    
    print('-------------------------------------------')

    # Print option price tree for American Call 
    print('Option Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2)))
    
    print('-------------------------------------------')

    # print('n = ' + str(n) + '):\n', np.matrix(Exer))
    
    

    # Return the option price at the root node
    return Call[0, 0]


In [40]:
# Parameters
S = 100          # Initial stock price
K = 101          # Strike price
v = 0.2          # Volatility
r = 0.05         # Risk-free interest rate
T = 2            # Time to maturity
n = 5            # Number of steps in the binomial tree
div = 0          # Dividend yield

# Calculate the binomial American put option price
Acp = BinoAmerCall(S, K, v, r, n, T, div)

# Print the result
print(f'The binomial American call price is {round(Acp, 3)}')


Stock Price Tree (n = 5):
 [[100.   113.48 128.79 146.15 165.86 188.22]
 [  0.    88.12 100.   113.48 128.79 146.15]
 [  0.     0.    77.65  88.12 100.   113.48]
 [  0.     0.     0.    68.42  77.65  88.12]
 [  0.     0.     0.     0.    60.29  68.42]
 [  0.     0.     0.     0.     0.    53.13]]
-------------------------------------------
Option Price Tree (n = 5):
 [[16.03 23.91 34.79 49.11 66.86 87.22]
 [ 0.    7.19 11.79 18.97 29.79 45.15]
 [ 0.    0.    1.94  3.6   6.71 12.48]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]
-------------------------------------------
The binomial American call price is 16.03


### Put Call parity : 
- long call, short put 

In [41]:
Acp - App

7.650184443961145

In [42]:
S - K * np.exp(-r * T)

8.611420778368085